# Načtení souborů
V tomto skriptu se načtou log soubory uložené v ZIP souboru. Tento soubor si stáhněte a nahrajte do stejného adresáře, jakou jsou tyto jupyter notebooky. Jedná se o evoluční návrh 4bitových sčítaček pro různé parametry lambda, počet sloupců a četnost mutace.

In [ ]:
! [ -e adders.zip ] || wget https://ehw.fit.vutbr.cz/bin/adders.zip
! [ -e example.log ] || wget https://ehw.fit.vutbr.cz/bin/example.log


In [ ]:
# Načtení potřebných knihoven
import zipfile
import pandas as pd
import re


V první funkci se pomocí regulárního výrazu naparsují vstupní parametry evoluce. Vytvoří se objekt typu `dictionary`. 

In [ ]:
def parse_filename(filename):
    # adders/FA-adder_full_u4bit-L1-C0050-M01-R0000.log
    g = re.match(r".*u(\d)bit-L(\d+)-C(\d+)-M(\d+)-R(\d+)\.log", filename)
    bw, lback, columns, mutations, run = g.groups()
    return {
        "bw": int(bw),
        "lambda_p": int(lback),
        "cols": int(columns),
        "muts": int(mutations),
        "run": int(run)
    }


parse_filename("adders/FA-adder_full_u4bit-L1-C0050-M01-R0000.log")


Další funkce projde všechny řádky souboru a poslední řádek určující výslednou kvalitu se určí počet generací, doba trvání (v sekundách), zda se podařilo najít funkční řešení a celkový počet uzlů.

In [ ]:
def parse_log(file):
    last_line = None
    # pruchod pres vsechny radky a hledani vystupu fitness
    # generation:         1 time: 0.000 idx:0 fitness: 1296/2560 active nodes: 26
    for l in file.readlines():
        if l.startswith("generation:"):
            last_line = l

    assert last_line

    g = re.match(
        r"generation:\s*(\d+) time:\s*(\d+\.\d+) idx:\d+ fitness: (\d+)/(\d+) active nodes: (\d+)", last_line)

    gen, time, fit, fitmax, active = g.groups()
    gen, fit, fitmax, active = int(gen), int(fit), int(fitmax), int(active)

    return {
        "gen": int(gen),
        "duration": float(time),
        "success": int(fit) >= int(fitmax),
        "nodes": int(active)
    }


print(parse_log(open("example.log", "r")))


Nyní se projodou všechny soubory a vytvoří se jeden slovník. Konstrukce `{**a, **b}` slouží ke spojení dvou slovníků.

In [ ]:
from io import TextIOWrapper
alld = []
with zipfile.ZipFile("adders.zip") as zf:
    for filename in zf.namelist():
        if not filename.endswith(".log"):
            continue
        d = {"filename": filename}
        d = {**d, **parse_filename(filename)}
        # IO wrapper -> prevod z bytu na str
        d = {**d, **parse_log(TextIOWrapper(zf.open(filename, "r")))}

        alld.append(d)
alld


Nyní se vytvoří z pole slovníků jeden dataframe (tabulka), se kterou budeme dále pracovat. 

In [ ]:
df = pd.DataFrame(alld)
df


S tabulkou můžeme dále pracovat - např. si určit úspěšnost návrhu.

In [ ]:
df["success"].mean()


A data si uložíme do Pickle souboru pro další zpracování.

In [ ]:
df.to_pickle("data.pkl.gz")


# Zpracování dat
Tento notebook představuje hlavní část, se kterou budete ve svém řešení projektu pracovat. Načtou se data, která jste vygenerovali v části 1 a budete je dále analyzovat. Normálně by toto byl samostatný soubor, nicméně pro práci s Google Colab je lepší pracovat s jedním notebookem

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np


Pomocí funkce `pd.read_pickle` načtěte soubor _data.pkl.gz_. Tento soubor uložte do proměnné `df` a zobrazte ji jako tabulku (tj. poslední řádek buňky bude `df`).

In [ ]:
# TODO


Pomocí funkce `unique()` vázané k sérii (sloupci) vypište unikátní hodnoty slouců _cols, muts, lambda_p_.

In [ ]:
# TODO


## Jednoduchá analýza výsledků
Pomocí filtrace `df.query` vytvořte nový dataframe `df2`, který bude obsahovat pouze výsledky s 5 mutacemi a parametrem _lambda_p_ roven 1 a s úspěšnými hledáními (_success_ je pozitivní). Tento dataframe rovnou zobrazte.

In [ ]:
# TODO


## Tvorba boxplotu
Vytvořte jednoduchý boxplot pro dobu trvání (sloupec _duration_) pro jednotlivé nastavení počtu sloupců (_cols_). Pro data pro každý ze tří boxplotů můžete použtí filtraci z `df2` příkazem `query`, ze kterého pak vyberete pouze duration - t.j. `df2.query("...")["duration]`. Nezapomeňte nastavit popisky os a ticky na ose X.

In [ ]:
# TODO


Do tohoto pole napište závěr z boxplotu. Co můžeme vidět? (cca 2 věty)

## Tvorba histogramu
Nyní si zanalyzujeme výsledky trvání pro __50 sloupců__ detailně. Vykreslete proto histogram pro toto nastavení. Použijte filtraci jako výše. 

Graf nezapomeňte správně nakonfigurovat (titulek, popisy os, limit a podobně)

In [ ]:
# TODO


## Statistická analýza
Jaká je průměrná doba trvání pro 100 a 500 sloupců? Vytvořte si dvě série výběrem jako výše a vypište jejich: průměr, medián, minimum, maximum a 1. a 3. kvartil.

In [ ]:
# TODO


Podle mediánů se ukázalo, že jedno řešení je lepší. Je to opravdu statisticky významné? Proveďte [Mann-Whitney U-test](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html) (`mannwhitneyu`), protože data nejsou normální. Určete, zda je tento rozdíl signifikantní (s věrností 0.95).

> The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y. It is often used as a test of difference in location between distributions.

In [ ]:
# TODO


Napište závěr analýzy

## Pro rychlé a zvídavé
Tento notebook představuje rychlokurz datové analýzy a nepředstavuje vždy nejlepší řešení. Zkuste si pak implementovat boxploty pomocí Seabornu (`sns.boxplot`), kdy vyplníte parametry `data` (dataframe), `x` (název sloupce na x-ové ose) a `y` (název sloupce na ose y). Navíc můžete použít i větší dataset a nastavit parametr `hue` pro podbarvení (např. počet mutací). Návratová hodnota je matplotib _axis_ (`ax`), který můžete použít k další konfiguraci

In [ ]:
import seaborn as sns
# ax = sns.boxplot(data=df.query("success and lback==1"), ...) # todo (jen ve volném čase)


# Konvergenční křivky
Pro vybranou konfiguraci (s 90 běhy) nás bude zajímat, jak vlastně vypadá vývoj fitness v čase. Zase by se jednalo o samostatný soubor, ale kvůli Google Colab stále pracujeme v jednom notebooku.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import re


## Získávání dat
Nyní chceme převést každý log do 2D numpy pole, kdy první první sloupec obsahuje generaci a druhý sloupec odpovídající fitness. Vaším úkolem bude doplnit regulární výraz pro načítání generace a fitness a po dvojicích jej vložit do pole `alld`. Pro testovací log soubor by měl vzniknout následující výstup:

```[[      1    1296]
 [     12    1352]
 [     26    1392]
...
 [1589571    2536]
 [1647531    2560]]```
 

In [ ]:
# TODO
def parse_gens(file): ###
    # pruchod pres vsechny radky a hledani vystupu fitness ###
    # generation:         1 time: 0.000 idx:0 fitness: 1296/2560 active nodes: 26 ###
    alld = [] ###
    for l in file.readlines(): ###
        pass ###
        # napiste vhodny regularni vyrat podle sablony nahore ziskavajici  dvojici generace a fitness ###
        # z tohoto udelame 2d numpy pole (staci pridat do pole alld) ###
        # alld.append([ int(gen), int(fit)]) ###
###
    return np.array(alld)  ###
print(parse_gens(open("example.log", "r"))) ###


Nyní vytvoříme list numpy 2D polí (pro každý log soubor) odpovídající nastavení _bw=4, lambda=1, cols=50 a muts=5_ (vybráno náhodně). Nemohli jsme udělat jedno 3D pole, jelikož každé hledání vyžaduje jiný počet kroků - třetí dimenze pak není konstantní.

In [ ]:
from io import TextIOWrapper
allgens = []

alld = []
with zipfile.ZipFile("adders.zip") as zf:
    for filename in zf.namelist():
        if not filename.endswith(".log") or "FA-adder_full_u4bit-L1-C0050-M05-R" not in filename:
            continue
        # IO wrapper -> prevod z bytu na str
        gens = parse_gens(TextIOWrapper(zf.open(filename, "r")))

        allgens.append(gens)
allgens


## Všechny křivky přes sebe
Vykreslete graf všech běhů. Pro každý běh `for g in allgens` vykreslete jednu čáru - ideálně použijte funkci `ax.step(x = ..., y = ...  ,where="post", alpha = 0.2)`. Alfa kanál umožní zobrazení více čar přes sebe. 

Grafu nastavte logaritmické měřítko na ose X.

In [ ]:
# TODO
fig, ax = plt.subplots() ###
##
for g in allgens: ###
    pass ###
    # vykreslete pomoci ax.step ###
# nastavte parametry (log. meritko, ylim od 0 do None, popisky os, ...)


Tato konvergenční křivka sice trochu pomůže, ale není to správná vizualizace tohoto děje. 

## Vykreslení konvergenční křivky
Mohli bychom si spočítat minimum, maximum a průměr a vykreslit pomocí `plt.fill_between`, ale pro nás je výhodnější využít knihovnu _seaborn_. Tato knihovna umožňuje jednoduše generovat grafy z Pandas dataframu. Výhodou je to, že je pak můžeme upravit s využitím našich znalostí z matplotlib. Pro více ukázek doporučuji zkontrolovat [galerii](https://seaborn.pydata.org/examples/index.html).

V první části převedeme všechna data na jeden dataframe, kde přidáme sloupec "runid".

In [ ]:
alldf = []

for rid, g in enumerate(allgens):
    d = pd.DataFrame(g, columns=["gen", "fit"])
    d["runid"] = rid
    alldf.append(d)
df = pd.concat(alldf, ignore_index=True)
df


Vykreslete nyní čárový graf pomocí funkce `sns.lineplot`. Argumenty budou `data=df, x="gen", y="fit"`.

Návratová hodnota této funkce je objekt _Axis_ `ax`. Nastavte mu stejné parametry, jako výše (logaritmické měřítko a podobně).

In [ ]:
# TODO


Tento graf ale není správně. Správně se měl spočítat tzv. confidence interval a mělo být vidět, jak se fitness mění v čase. Problémem je to, že nemáme pro každou generaci definovaný výsledek pro všechny běhy. Nezbývá určit vzorkovací body, pro které si spočítáme (interpolujeme) hodnotu fitness.

Vytvořte tedy numpy pole `gens_selected` obsahující logaritmicky vzorkované body. Pro generování tohoto rozsahu můžete využít funkci `np.geomspace(start, stop, steps)` s tím, že zvolíte na základě předchozího grafu počátek, konec a vhodný počet bodů.

In [ ]:
# TODO
gens_selected ###

Nyní budeme interpolovat a extrapolovat. Využijeme k tomu funkci `scipy.interpolate.interp1d`. Tato funkce vytváří objekt na základě dvojic _x_ a _y_. Potom tento objekt můžeme zavolat (jako funkci) s libovolným vstupem (i vektorem) a vrátí nám body, které by fitness měla pro danou generaci. Normálně se dělá lineární proložení, nicméně zde budeme prokládat předchozí hodnotou (fitness se mění skokově).

In [ ]:
from scipy.interpolate import interp1d
alldf = []

for rid, g in enumerate(allgens):
    # vytvorime interpolacni funkci. Pridame i bod 0, protoze ten v seznamu neni
    g = np.concatenate([[[0, 0]], g])
    fn = interp1d(x=g[:, 0], y=g[:, 1], kind="previous",
                  fill_value="extrapolate")

    d = pd.DataFrame()
    # generace nastavime nasim vzorkovacim
    d["gen"] = gens_selected
    # a fitness odpovida prolozene hodnote
    d["fit"] = fn(gens_selected)
    d["runid"] = rid
    alldf.append(d)
df_interp = pd.concat(alldf, ignore_index=True)
df_interp


Nyní vykreslete graf stejně jako výše pomocí finkce `sns.lineplot`. Použijte funkci úplně stejně jako výše, jen místo `df` použijte naše nová vzrokovaná data `df_interp`. 

Vykreslil se už graf správně? Všimněte si, kde docházelo k variacím a kde už je velká jistota, že fitness bude vysoká. Je zadaný algoritmus stabilní z pohledu konvergence?

In [ ]:
# TODO


Barevný pruh nám určuje confidence interval (0.95) počítaný z distribuce. Střed je průměrná hodnota. My si však můžeme zobrazit např medián a minimum a maximum. Více v [dokumentaci](https://seaborn.pydata.org/tutorial/error_bars.html)

Zkuste přidat parametr `estimator=np.median`. V novějších verzích knihovny Seaborn (není případ Google Colab 🥹) se zadáva `estimator="median", errorbar=("pi", 100)` (100. percentil - min / max, můžete použít třeba 75.).

In [ ]:
# TODO
